In [1]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook as tqdm
from collections import Counter

import lightgbm

import seaborn as sn
import matplotlib.pyplot as plt
%matplotlib inline


# Загрузка данных

In [2]:
#DATA_DIR = 'data'   # Home
PROCESSED_DATA_DIR = 'processed/557'  # Work
#DATA_DIR = '../input/data-science-bowl-2019' # Kaggle

RANDOM_SEED = 17

In [3]:
import pickle
import os

featureset = None
with open(os.path.join(PROCESSED_DATA_DIR, 'train_processed.pkl'), 'rb') as fi:
    train = pickle.load(fi)

train.shape

(17687, 550)

In [4]:
from sklearn.model_selection import cross_val_score, GroupShuffleSplit
from sklearn.metrics import cohen_kappa_score

# Определение требуемой метрики 'quadratic weighted kappa'
scorer = lambda estimator, x, y : cohen_kappa_score(estimator.predict(x), y, weights='quadratic')

## Регрессия LightGBM с выбором порогов

In [5]:
from numba import jit

@jit
def qwk(a1, a2):
    """
    Source: https://www.kaggle.com/c/data-science-bowl-2019/discussion/114133#latest-660168

    :param a1:
    :param a2:
    :param max_rat:
    :return:
    """
    max_rat = 3
    a1 = np.asarray(a1, dtype=int)
    a2 = np.asarray(a2, dtype=int)

    hist1 = np.zeros((max_rat + 1, ))
    hist2 = np.zeros((max_rat + 1, ))

    o = 0
    for k in range(a1.shape[0]):
        i, j = a1[k], a2[k]
        hist1[i] += 1
        hist2[j] += 1
        o +=  (i - j) * (i - j)

    e = 0
    for i in range(max_rat + 1):
        for j in range(max_rat + 1):
            e += hist1[i] * hist2[j] * (i - j) * (i - j)

    e = e / a1.shape[0]

    return 1 - o / e


In [6]:
from functools import partial
import scipy as sp

class OptimizedRounder(object):
    """
    An optimizer for rounding thresholds
    to maximize Quadratic Weighted Kappa (QWK) score
    # https://www.kaggle.com/naveenasaithambi/optimizedrounder-improved
    """
    def __init__(self):
        self.coef_ = 0

    def _kappa_loss(self, coef, X, y):
        """
        Get loss according to
        using current coefficients
        
        :param coef: A list of coefficients that will be used for rounding
        :param X: The raw predictions
        :param y: The ground truth labels
        """
        X_p = pd.cut(X, [-np.inf] + list(np.sort(coef)) + [np.inf], labels = [0, 1, 2, 3])

        return -qwk(y, X_p)

    def fit(self, X, y):
        """
        Optimize rounding thresholds
        
        :param X: The raw predictions
        :param y: The ground truth labels
        """
        loss_partial = partial(self._kappa_loss, X=X, y=y)
        initial_coef = [0.5, 1.5, 2.5]
        self.coef_ = sp.optimize.minimize(loss_partial, initial_coef, method='nelder-mead')

    def predict(self, X, coef):
        """
        Make predictions with specified thresholds
        
        :param X: The raw predictions
        :param coef: A list of coefficients that will be used for rounding
        """
        return pd.cut(X, [-np.inf] + list(np.sort(coef)) + [np.inf], labels = [0, 1, 2, 3])


    def coefficients(self):
        """
        Return the optimized coefficients
        """
        return self.coef_['x']
    
    
class MultistartOptimizedRounder(object):
    """
    An optimizer for rounding thresholds
    to maximize Quadratic Weighted Kappa (QWK) score
    # https://www.kaggle.com/naveenasaithambi/optimizedrounder-improved
    """
    def __init__(self):
        self.coef_ = 0

    def _kappa_loss(self, coef, X, y):
        """
        Get loss according to
        using current coefficients
        
        :param coef: A list of coefficients that will be used for rounding
        :param X: The raw predictions
        :param y: The ground truth labels
        """
        X_p = pd.cut(X, [-np.inf] + list(np.sort(coef)) + [np.inf], labels = [0, 1, 2, 3])

        return -qwk(y, X_p)

    def fit(self, X, y):
        """
        Optimize rounding thresholds
        
        :param X: The raw predictions
        :param y: The ground truth labels
        """
        loss_partial = partial(self._kappa_loss, X=X, y=y)
        initial_coefs = [[0.5, 1.5, 2.5],
                         [1.1, 1.6, 2.2],
                         [1.2, 1.7, 2.1]]
        best_val = 10.0                    # значение берется с обратным знаком
        best_coef = None
        for coef in initial_coefs:
            cur_coef = sp.optimize.minimize(loss_partial, coef, method='nelder-mead')
            cur_val = self._kappa_loss(cur_coef['x'], X, y)
            print('Val:', cur_val)
            if cur_val < best_val:         # значение берется с обратным знаком
                print('New best!')
                best_val = cur_val
                best_coef = cur_coef
        self.coef_ = best_coef

    def predict(self, X, coef):
        """
        Make predictions with specified thresholds
        
        :param X: The raw predictions
        :param coef: A list of coefficients that will be used for rounding
        """
        return pd.cut(X, [-np.inf] + list(np.sort(coef)) + [np.inf], labels = [0, 1, 2, 3])


    def coefficients(self):
        """
        Return the optimized coefficients
        """
        return self.coef_['x']

In [7]:
from sklearn.base import BaseEstimator, MetaEstimatorMixin, ClassifierMixin, clone

class ClassifierWrapper(BaseEstimator, MetaEstimatorMixin, ClassifierMixin):
    
    def __init__(self, base_estimator, estimator_params):
        super().__init__()
        self.base_estimator = base_estimator
        self.estimator_params = estimator_params        
        self.optimized_rounder = MultistartOptimizedRounder()

    def make_estimator(self):
        estimator = clone(self.base_estimator)
        #estimator.set_params(**dict((p, getattr(self, p))
        #                            for p in self.estimator_params))
        estimator.set_params(**self.estimator_params)
        return estimator        
        
    def fit(self, X, y):
        self.model = self.make_estimator().fit(X, y)
        y_pred = self.model.predict(X)
        self.optimized_rounder.fit(y_pred.reshape(-1,), y)
        # coefficients = optR.coefficients()
        print('Found bounds:', self.optimized_rounder.coefficients())
        
    def predict(self, X):
        y_pred = self.model.predict(X)
        return self.optimized_rounder.predict(y_pred.reshape(-1, ), self.optimized_rounder.coefficients())
        

In [8]:
X = train.drop(columns=['installation_id', 'timestamp', 
                        'accuracy_group', 'accuracy', 'num_correct', 'num_incorrect'])
y = train[['accuracy_group']]


In [9]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import GroupKFold

# Определение требуемой метрики 'quadratic weighted kappa'
scorer = lambda estimator, x, y : cohen_kappa_score(estimator.predict(x), y, weights='quadratic')

In [10]:
import random
from sklearn.model_selection import KFold

class OneTestPerGroupKFold:
    """
    Генерация фолдов для кросс-валидации, в которых в тестовой части будет
    только по одной записи от каждого пользователя (installation_id).
    """
    
    def __init__(self, n_splits=5, shuffle=False, random_state=None):
        self.n_splits = n_splits
        if shuffle:
            self.splitter = KFold(n_splits, shuffle=True, random_state=random_state)
        else:
            self.splitter = KFold(n_splits)
        if random_state is None:
            random_state = 0
        self.random = random.Random(random_state)
        
    def get_n_splits(self):
        return self.n_splits
    
    def split(self, X, groups):
        # Составим вспомогательный набор с отображением атрибута
        # группировки (installation_id) на порядковый индекс 
        # соответствующей записи
        tt = pd.DataFrame({'group': groups, 'idx': np.arange(groups.shape[0])})
        # Выбираем по одному представителю для каждой группы
        picks = tt.groupby(['group'], sort=False)['idx'].agg(lambda x: x.iloc[self.random.randint(0, x.shape[0]-1)])
        # Генерируем фолды по группам (пользователям)
        for train_idx, test_idx in self.splitter.split(picks):
            # "Раскроем" индексы:
            # Для обучающей выборки это будут все записи соответствующих групп
            train_idx_ = tt.idx[tt.group.isin(list(picks.iloc[train_idx].index))].to_numpy(dtype='int')
            # Для тестовой выборки это будут только выбранные записи из соответствующих групп
            test_idx_ = picks.iloc[test_idx].to_numpy(dtype='int')
            yield (train_idx_, test_idx_)

In [11]:
X.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in X.columns]

# Baseline: модель с исходными параметрами

In [12]:

# from Andrew Lukyanenko
parameters = {'n_estimators':2000,
            'boosting_type': 'gbdt',
            'objective': 'regression',
            'metric': 'rmse',
            'subsample': 0.75,
            'subsample_freq': 1,
            'learning_rate': 0.04,
            'feature_fraction': 0.9,
         'max_depth': 15,
            'lambda_l1': 1,  
            'lambda_l2': 1,
            'verbose': 100,
            'random_state': RANDOM_SEED,
            'n_jobs': 16,
            'early_stopping_rounds': 100, 'eval_metric': 'cappa'
            }

splitter = OneTestPerGroupKFold(5, shuffle=True, random_state=RANDOM_SEED)

scores = []
for train_idxs, test_idxs in splitter.split(X, groups=train.installation_id):
    train_X = X.iloc[train_idxs, :]
    train_y = y.iloc[train_idxs, :]
    test_X = X.iloc[test_idxs, :]
    test_y = y.iloc[test_idxs, :]
    train_data = lightgbm.Dataset(train_X, label=train_y, categorical_feature=['session_title', 'world'])
    test_data = lightgbm.Dataset(test_X, label=test_y)
    model = lightgbm.train(parameters,
                           train_data,
                           valid_sets=test_data)
    rounder = MultistartOptimizedRounder()
    rounder.fit(model.predict(train_X).reshape(-1), train_y)
    y_pred = rounder.predict(model.predict(test_X).reshape(-1, ), rounder.coefficients())
    scores.append(cohen_kappa_score(y_pred, test_y, weights='quadratic'))

scores = np.array(scores)    
print(scores)    
print(scores.mean())    
print(scores.std())    
print(scores.min())    
    


/home/paniquex/anaconda3/envs/kaggle/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/paniquex/anaconda3/envs/kaggle/lib/python3.7/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/paniquex/anaconda3/envs/kaggle/lib/python3.7/site-packages/lightgbm/basic.py:1291: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/home/paniquex/anaconda3/envs/kaggle/lib/python3.7/site-packages/lightgbm/basic.py:1295: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['session_title', 'world']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


[1]	valid_0's rmse: 1.2793
Training until validation scores don't improve for 100 rounds
[2]	valid_0's rmse: 1.26636
[3]	valid_0's rmse: 1.254
[4]	valid_0's rmse: 1.24265
[5]	valid_0's rmse: 1.2311
[6]	valid_0's rmse: 1.22478
[7]	valid_0's rmse: 1.21473
[8]	valid_0's rmse: 1.20835
[9]	valid_0's rmse: 1.19984
[10]	valid_0's rmse: 1.19106
[11]	valid_0's rmse: 1.18392
[12]	valid_0's rmse: 1.17907
[13]	valid_0's rmse: 1.17231
[14]	valid_0's rmse: 1.16538
[15]	valid_0's rmse: 1.1598
[16]	valid_0's rmse: 1.15429
[17]	valid_0's rmse: 1.14851
[18]	valid_0's rmse: 1.14336
[19]	valid_0's rmse: 1.13863
[20]	valid_0's rmse: 1.13406
[21]	valid_0's rmse: 1.12904
[22]	valid_0's rmse: 1.12448
[23]	valid_0's rmse: 1.12052
[24]	valid_0's rmse: 1.11652
[25]	valid_0's rmse: 1.11325
[26]	valid_0's rmse: 1.11002
[27]	valid_0's rmse: 1.10656
[28]	valid_0's rmse: 1.10304
[29]	valid_0's rmse: 1.10078
[30]	valid_0's rmse: 1.09891
[31]	valid_0's rmse: 1.09696
[32]	valid_0's rmse: 1.09392
[33]	valid_0's rmse: 1.0

<ipython-input-5-7e6567382b89>:3: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "qwk" failed type inference due to: non-precise type pyobject
[1] During: typing of argument at <ipython-input-5-7e6567382b89> (13)

File "<ipython-input-5-7e6567382b89>", line 13:
def qwk(a1, a2):
    <source elided>
    """
    max_rat = 3
    ^

  @jit
<ipython-input-5-7e6567382b89>:3: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "qwk" failed type inference due to: cannot determine Numba type of <class 'numba.dispatcher.LiftedLoop'>

File "<ipython-input-5-7e6567382b89>", line 21:
def qwk(a1, a2):
    <source elided>
    o = 0
    for k in range(a1.shape[0]):
    ^

  @jit
/home/paniquex/anaconda3/envs/kaggle/lib/python3.7/site-packages/numba/object_mode_passes.py:178: NumbaWarning: Function "qwk" was compiled in object mode without forceobj=True, but has lifted loops.

File "<ipython-input-5-7

Val: [-0.71367074]
New best!
Val: [-0.7326435]
New best!
Val: [-0.73132435]


/home/paniquex/anaconda3/envs/kaggle/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/paniquex/anaconda3/envs/kaggle/lib/python3.7/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/paniquex/anaconda3/envs/kaggle/lib/python3.7/site-packages/lightgbm/basic.py:1291: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/home/paniquex/anaconda3/envs/kaggle/lib/python3.7/site-packages/lightgbm/basic.py:1295: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['session_title', 'world']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


[1]	valid_0's rmse: 1.23193
Training until validation scores don't improve for 100 rounds
[2]	valid_0's rmse: 1.21868
[3]	valid_0's rmse: 1.20597
[4]	valid_0's rmse: 1.19371
[5]	valid_0's rmse: 1.1821
[6]	valid_0's rmse: 1.17519
[7]	valid_0's rmse: 1.16517
[8]	valid_0's rmse: 1.1591
[9]	valid_0's rmse: 1.14982
[10]	valid_0's rmse: 1.14151
[11]	valid_0's rmse: 1.13353
[12]	valid_0's rmse: 1.12875
[13]	valid_0's rmse: 1.12175
[14]	valid_0's rmse: 1.11546
[15]	valid_0's rmse: 1.1097
[16]	valid_0's rmse: 1.10409
[17]	valid_0's rmse: 1.09899
[18]	valid_0's rmse: 1.09373
[19]	valid_0's rmse: 1.0896
[20]	valid_0's rmse: 1.08569
[21]	valid_0's rmse: 1.08151
[22]	valid_0's rmse: 1.07724
[23]	valid_0's rmse: 1.07429
[24]	valid_0's rmse: 1.07073
[25]	valid_0's rmse: 1.06804
[26]	valid_0's rmse: 1.06546
[27]	valid_0's rmse: 1.06196
[28]	valid_0's rmse: 1.05914
[29]	valid_0's rmse: 1.05664
[30]	valid_0's rmse: 1.05374
[31]	valid_0's rmse: 1.05179
[32]	valid_0's rmse: 1.04976
[33]	valid_0's rmse: 1.

[54]	valid_0's rmse: 1.02891
[55]	valid_0's rmse: 1.02812
[56]	valid_0's rmse: 1.02703
[57]	valid_0's rmse: 1.02601
[58]	valid_0's rmse: 1.02548
[59]	valid_0's rmse: 1.02485
[60]	valid_0's rmse: 1.02441
[61]	valid_0's rmse: 1.02397
[62]	valid_0's rmse: 1.02354
[63]	valid_0's rmse: 1.02301
[64]	valid_0's rmse: 1.02161
[65]	valid_0's rmse: 1.02087
[66]	valid_0's rmse: 1.02025
[67]	valid_0's rmse: 1.01982
[68]	valid_0's rmse: 1.01908
[69]	valid_0's rmse: 1.01846
[70]	valid_0's rmse: 1.01808
[71]	valid_0's rmse: 1.01747
[72]	valid_0's rmse: 1.01747
[73]	valid_0's rmse: 1.01717
[74]	valid_0's rmse: 1.01714
[75]	valid_0's rmse: 1.01691
[76]	valid_0's rmse: 1.0169
[77]	valid_0's rmse: 1.01704
[78]	valid_0's rmse: 1.01663
[79]	valid_0's rmse: 1.01639
[80]	valid_0's rmse: 1.01631
[81]	valid_0's rmse: 1.01589
[82]	valid_0's rmse: 1.01552
[83]	valid_0's rmse: 1.01519
[84]	valid_0's rmse: 1.015
[85]	valid_0's rmse: 1.0148
[86]	valid_0's rmse: 1.01491
[87]	valid_0's rmse: 1.01471
[88]	valid_0's rms

[89]	valid_0's rmse: 1.02922
[90]	valid_0's rmse: 1.02893
[91]	valid_0's rmse: 1.02865
[92]	valid_0's rmse: 1.02876
[93]	valid_0's rmse: 1.02837
[94]	valid_0's rmse: 1.02841
[95]	valid_0's rmse: 1.02831
[96]	valid_0's rmse: 1.0277
[97]	valid_0's rmse: 1.02776
[98]	valid_0's rmse: 1.02751
[99]	valid_0's rmse: 1.02744
[100]	valid_0's rmse: 1.02737
[101]	valid_0's rmse: 1.02738
[102]	valid_0's rmse: 1.02727
[103]	valid_0's rmse: 1.02679
[104]	valid_0's rmse: 1.02661
[105]	valid_0's rmse: 1.02685
[106]	valid_0's rmse: 1.02688
[107]	valid_0's rmse: 1.02664
[108]	valid_0's rmse: 1.02639
[109]	valid_0's rmse: 1.02617
[110]	valid_0's rmse: 1.02632
[111]	valid_0's rmse: 1.02657
[112]	valid_0's rmse: 1.02641
[113]	valid_0's rmse: 1.02613
[114]	valid_0's rmse: 1.02581
[115]	valid_0's rmse: 1.02546
[116]	valid_0's rmse: 1.02544
[117]	valid_0's rmse: 1.02553
[118]	valid_0's rmse: 1.02567
[119]	valid_0's rmse: 1.02572
[120]	valid_0's rmse: 1.02584
[121]	valid_0's rmse: 1.02562
[122]	valid_0's rmse: 

[372]	valid_0's rmse: 1.02289
[373]	valid_0's rmse: 1.02289
[374]	valid_0's rmse: 1.02283
[375]	valid_0's rmse: 1.02264
[376]	valid_0's rmse: 1.02282
[377]	valid_0's rmse: 1.02287
[378]	valid_0's rmse: 1.023
[379]	valid_0's rmse: 1.02307
[380]	valid_0's rmse: 1.023
[381]	valid_0's rmse: 1.02292
[382]	valid_0's rmse: 1.02289
[383]	valid_0's rmse: 1.02319
[384]	valid_0's rmse: 1.02295
[385]	valid_0's rmse: 1.02302
[386]	valid_0's rmse: 1.02326
[387]	valid_0's rmse: 1.02324
[388]	valid_0's rmse: 1.02303
[389]	valid_0's rmse: 1.0232
[390]	valid_0's rmse: 1.02303
[391]	valid_0's rmse: 1.02313
Early stopping, best iteration is:
[291]	valid_0's rmse: 1.02134
Val: [-0.78480286]
New best!
Val: [-0.78501548]
New best!
Val: [-0.78489192]
[1]	valid_0's rmse: 1.27436
Training until validation scores don't improve for 100 rounds
[2]	valid_0's rmse: 1.26026
[3]	valid_0's rmse: 1.24662
[4]	valid_0's rmse: 1.23401
[5]	valid_0's rmse: 1.22304
[6]	valid_0's rmse: 1.21555
[7]	valid_0's rmse: 1.20498
[8]	v

Val: [-0.72307638]
New best!
Val: [-0.7226984]
Val: [-0.72258195]
[0.57002833 0.55752878 0.58118632 0.55350625 0.58138832]
0.5687276006759764
0.011612997348712961
0.5535062493087048


[0.59518972 0.57949122 0.59924864 0.63245339 0.58247959]

0.5977725137414479
0.01886633180354496

[0.56387506 0.55242486 0.57958958 0.53439198 0.58543279]

0.5631428545007849
0.018489781426196183
0.53439198072543

# Настройка параметров

In [13]:
import optuna

In [17]:
def evaluate_parameters(params):
    scores = []
    splitter = OneTestPerGroupKFold(5, shuffle=True, random_state=RANDOM_SEED)
    for train_idxs, test_idxs in splitter.split(X, groups=train.installation_id):
        train_X = X.iloc[train_idxs, :]
        train_y = y.iloc[train_idxs, :]
        test_X = X.iloc[test_idxs, :]
        test_y = y.iloc[test_idxs, :]
        train_data = lightgbm.Dataset(train_X, label=train_y, categorical_feature=['session_title', 'world'])
        test_data = lightgbm.Dataset(test_X, label=test_y)
        model = lightgbm.train(params,
                               train_data,
                               valid_sets=test_data,
                               verbose_eval=False)
        rounder = MultistartOptimizedRounder()
        rounder.fit(model.predict(train_X).reshape(-1), train_y)
        y_pred = rounder.predict(model.predict(test_X).reshape(-1, ), rounder.coefficients())
        scores.append(cohen_kappa_score(y_pred, test_y, weights='quadratic'))

    scores = np.array(scores)    
    print(scores)    
    print(scores.mean())    
    print(scores.std())
    print(scores.min())    
    
    return scores.mean()

def objective(trial):
    param = {
        'n_estimators':2000,
        'boosting_type': 'gbdt',
        'objective': 'regression',
        'metric': 'rmse',
        'verbose': 0,
        'random_state': RANDOM_SEED,
        'early_stopping_rounds': 100,
        'n_jobs': 16,
        
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-2, 10.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-2, 10.0),
        'num_leaves': trial.suggest_int('num_leaves', 16, 128),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.5, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.5, 1.0),     # subsample   
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 3),                     # subsample_freq
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'learning_rate': trial.suggest_loguniform('learning_rate', 7e-3, 0.32),
    }

    # TODO pruning callbacks 
    return evaluate_parameters(param)
    

In [18]:
study = optuna.create_study(study_name='557_LGBM_Tuning_ONE_MIN', direction='maximize', storage='sqlite:///557_lgbm_tuning_OPU_min.db')
study.optimize(objective, n_trials = 100)

[I 2020-01-20 16:57:32,445] A new study created with name: 557_LGBM_Tuning_ONE_MIN
/home/paniquex/anaconda3/envs/kaggle/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning:

Found `n_estimators` in params. Will use it instead of argument

/home/paniquex/anaconda3/envs/kaggle/lib/python3.7/site-packages/lightgbm/engine.py:153: UserWarning:

Found `early_stopping_rounds` in params. Will use it instead of argument

/home/paniquex/anaconda3/envs/kaggle/lib/python3.7/site-packages/lightgbm/basic.py:1291: UserWarning:

Using categorical_feature in Dataset.

/home/paniquex/anaconda3/envs/kaggle/lib/python3.7/site-packages/lightgbm/basic.py:1295: UserWarning:

categorical_feature in Dataset is overridden.
New categorical_feature is ['session_title', 'world']



Val: [-0.749356]
New best!
Val: [-0.77179905]
New best!
Val: [-0.77179905]
Val: [-0.77772906]
New best!
Val: [-0.77773757]
New best!
Val: [-0.77766578]
Val: [-0.72086961]
New best!
Val: [-0.72067964]
Val: [-0.72100902]
New best!
Val: [-0.7524834]
New best!
Val: [-0.75220696]
Val: [-0.75226692]
Val: [-0.76806681]
New best!
Val: [-0.76926227]
New best!
Val: [-0.76867108]
[0.57635639 0.55737446 0.58820815 0.54066939 0.56827711]
0.5661771010102451
0.016257462421728847
0.540669385978921


[I 2020-01-20 17:00:33,165] Finished trial#0 resulted in value: 0.5661771010102451. Current best value is 0.5661771010102451 with parameters: {'bagging_fraction': 0.7692253148376236, 'bagging_freq': 2, 'feature_fraction': 0.9598109021467595, 'lambda_l1': 4.7530471936334, 'lambda_l2': 0.1158402647851884, 'learning_rate': 0.05320047398092509, 'max_depth': 9, 'min_child_samples': 37, 'num_leaves': 54}.


Val: [-0.68853838]
New best!
Val: [-0.68810746]
Val: [-0.68804347]
Val: [-0.69869245]
New best!
Val: [-0.69795306]
Val: [-0.69767042]
Val: [-0.67758462]
New best!
Val: [-0.68037841]
New best!
Val: [-0.68066411]
New best!
Val: [-0.67029319]
New best!
Val: [-0.67097607]
New best!
Val: [-0.67065883]
Val: [-0.7264658]
New best!
Val: [-0.72790809]
New best!
Val: [-0.72652658]
[0.56999692 0.5519389  0.56841027 0.53907279 0.57932598]
0.5617489731873899
0.014365231541129444
0.5390727946864413


[I 2020-01-20 17:03:28,635] Finished trial#1 resulted in value: 0.5617489731873899. Current best value is 0.5661771010102451 with parameters: {'bagging_fraction': 0.7692253148376236, 'bagging_freq': 2, 'feature_fraction': 0.9598109021467595, 'lambda_l1': 4.7530471936334, 'lambda_l2': 0.1158402647851884, 'learning_rate': 0.05320047398092509, 'max_depth': 9, 'min_child_samples': 37, 'num_leaves': 54}.


...

[I 2020-01-21 00:53:35,356] Finished trial#97 resulted in value: 0.5680450176693295. Current best value is 0.5783426706344399 with parameters: {'bagging_fraction': 0.7236140037191053, 'bagging_freq': 1, 'feature_fraction': 0.5727800352800161, 'lambda_l1': 3.6408381403577934, 'lambda_l2': 0.09829070667029906, 'learning_rate': 0.012695180807095679, 'max_depth': 7, 'min_child_samples': 34, 'num_leaves': 57}.


Val: [-0.7776237]
New best!
Val: [-0.77738703]
Val: [-0.77727177]
Val: [-0.74693997]
New best!
Val: [-0.74653694]
Val: [-0.74639092]
Val: [-0.76076591]
New best!
Val: [-0.7604525]
Val: [-0.76025376]
Val: [-0.76269489]
New best!
Val: [-0.76374125]
New best!
Val: [-0.76438489]
New best!
Val: [-0.74334483]
New best!
Val: [-0.74409932]
New best!
Val: [-0.744247]
New best!
[0.57675427 0.54972283 0.5846071  0.5473411  0.58321199]
0.5683274596768929
0.016396015431548755
0.5473411048999043


[I 2020-01-21 00:57:00,294] Finished trial#98 resulted in value: 0.5683274596768929. Current best value is 0.5783426706344399 with parameters: {'bagging_fraction': 0.7236140037191053, 'bagging_freq': 1, 'feature_fraction': 0.5727800352800161, 'lambda_l1': 3.6408381403577934, 'lambda_l2': 0.09829070667029906, 'learning_rate': 0.012695180807095679, 'max_depth': 7, 'min_child_samples': 34, 'num_leaves': 57}.


Val: [-0.76275669]
New best!
Val: [-0.76921396]
New best!
Val: [-0.76903917]
Val: [-0.74237838]
New best!
Val: [-0.74351658]
New best!
Val: [-0.74215886]
Val: [-0.75099818]
New best!
Val: [-0.75115395]
New best!
Val: [-0.75081216]
Val: [-0.75667368]
New best!
Val: [-0.76248708]
New best!
Val: [-0.76251715]
New best!
Val: [-0.75541948]
New best!
Val: [-0.75556282]
New best!
Val: [-0.75485651]
[0.57932837 0.55727513 0.58478379 0.56416425 0.57304418]
0.5717191438432538
0.009960389037484708
0.557275131074479


[I 2020-01-21 01:00:21,492] Finished trial#99 resulted in value: 0.5717191438432538. Current best value is 0.5783426706344399 with parameters: {'bagging_fraction': 0.7236140037191053, 'bagging_freq': 1, 'feature_fraction': 0.5727800352800161, 'lambda_l1': 3.6408381403577934, 'lambda_l2': 0.09829070667029906, 'learning_rate': 0.012695180807095679, 'max_depth': 7, 'min_child_samples': 34, 'num_leaves': 57}.
